In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import shutil
import cobra as cb
from joblib import Parallel, delayed
from tqdm import tqdm
import logging
import glob

#### processsamento dei dati

Convertire adata  di flussi in csv

In [2]:
adata = sc.read_h5ad("../dati_sampling/dati/sampling/CBS/mean")
cbs_df = pd.DataFrame(data=adata.X, index=adata.obs_names, columns=adata.var_names)

In [4]:
cbs_df.to_csv("cbs_mean.csv")

Spostare il file di sampling totale in un'unica cartella

In [4]:
main_folder = '../dati_sampling/dati/sampling/CBS/'

destination_folder = '../dati_sampling/dati/sampling/CBS_total/'
os.makedirs(destination_folder, exist_ok=True)

# Ottieni l'elenco delle sottocartelle
subfolders = [subfolder for subfolder in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, subfolder))]

for subfolder in tqdm(subfolders, desc="Spostamento file"):
    subfolder_path = os.path.join(main_folder, subfolder)

    # Crea il percorso del file .pkl nella sottocartella
    file_path = os.path.join(subfolder_path, f'{subfolder}.pkl')

    # Verifica se il file .pkl esiste
    if os.path.isfile(file_path):
        # Sposta il file nella cartella di destinazione
        shutil.move(file_path, destination_folder)
    else:
        tqdm.write(f'File non trovato: {file_path}')

Spostamento file: 0it [00:00, ?it/s]


#### Iterazione per determinare lo stato di TCA

In [ ]:
def count_tca_states(df):
    """
    Analizza le righe di un DataFrame per determinare lo stato di attivazione del ciclo TCA
    canonico e non canonico, basato su criteri specificati. Restituisce i conteggi dei campionamenti
    per biomassa, TCA canonico, TCA non canonico e entrambi attivi, insieme alle medie di biomassa
    e misure di attività basate sui valori assoluti minimi per ciascun stato.

    :param df: DataFrame con i dati
    :return: Un dizionario con i conteggi, medie di biomassa e misure di attività dei campionamenti
    """

    # Criteri per determinare lo stato di attivazione di TCA canonico
    def is_tca_canonico(row):
        return (row['ACONTm'] > 1e-6) and \
               (row['ICDHym'] > 1e-6 or row['ICDHxm'] > 1e-6) and \
               (row['AKGDm'] > 1e-6) and \
               (row['SUCOAS1m'] > 1e-6) and \
               (row['SUCD1m'] > 1e-6) and \
               (row['FUMm'] > 1e-6) and \
               (row['MDHm'] > 1e-6) and \
               (row['CSm'] > 1e-6)

    # Criteri per determinare lo stato di attivazione di TCA non canonico
    def is_tca_non_canonico(row):
        return (row['HMR_4964'] > 1e-6) and \
               (row['ACITL'] > 1e-6) and \
               (row['MDH'] < -1e-6)

    # Applica i criteri per determinare lo stato di attivazione
    is_canonico = df.apply(is_tca_canonico, axis=1)
    is_non_canonico = df.apply(is_tca_non_canonico, axis=1)

    # Calcolare il numero di campionamenti per ciascuno stato
    num_tca_canonico = is_canonico.sum()
    num_tca_non_canonico = is_non_canonico.sum()
    num_both_active = (is_canonico & is_non_canonico).sum()

    # Medie di biomassa per ciascuno stato
    mean_biomass_tca_canonico = df[is_canonico]['Biomass'].mean() if num_tca_canonico > 0 else None
    mean_biomass_tca_non_canonico = df[is_non_canonico]['Biomass'].mean() if num_tca_non_canonico > 0 else None
    mean_biomass_both_active = df[is_canonico & is_non_canonico]['Biomass'].mean() if num_both_active > 0 else None

    # Valore assoluto minimo per ciascun stato
    def calculate_min_abs_activity(df, criteria, columns):
        return df[criteria].apply(lambda row: row[columns].abs().min(), axis=1)

    tca_canonico_columns = ['ACONTm', 'ICDHym', 'ICDHxm', 'AKGDm', 'SUCOAS1m', 'SUCD1m', 'FUMm', 'MDHm', 'CSm']
    tca_non_canonico_columns = ['HMR_4964', 'ACITL', 'MDH']

    min_abs_activity_tca_canonico = calculate_min_abs_activity(df, is_canonico, tca_canonico_columns)
    min_abs_activity_tca_non_canonico = calculate_min_abs_activity(df, is_non_canonico, tca_non_canonico_columns)

    mean_min_abs_activity_tca_canonico = min_abs_activity_tca_canonico.mean() if num_tca_canonico > 0 else None
    mean_min_abs_activity_tca_non_canonico = min_abs_activity_tca_non_canonico.mean() if num_tca_non_canonico > 0 else None

    # Restituisce i risultati
    return {
        'TCA canonical': num_tca_canonico,
        'TCA non canonical': num_tca_non_canonico, # <----- cycle propensity
        'both active': num_both_active,
        'biomass mean canonical': mean_biomass_tca_canonico,
        'biomass mean non canonical': mean_biomass_tca_non_canonico,
        'biomass mean both active': mean_biomass_both_active,
        'activity mean TCA canonical': mean_min_abs_activity_tca_canonico,
        'activity mean TCA non canonical': mean_min_abs_activity_tca_non_canonico # <----- cycle flux intensity
    }


In [3]:
# determinare lo stato di attivazione del ciclo TCA canonico e non canonico per ciascun cellula
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def process_file(file_path):
    try:
        df = pd.read_pickle(file_path)
        results = count_tca_states(df)
        file_name_ex = os.path.basename(file_path)
        file_name, _ = os.path.splitext(file_name_ex)
        results['Cell_lines'] = file_name
        
        logging.info(f'Processed {file_name}')
        return results
    except Exception as e:
        logging.error(f'Error processing {file_path}: {e}')
        return None

file_paths = glob.glob('../dati_sampling/dati/sampling/CBS_total/*.pkl')

# Parallelizzazione del processo di analisi
results_list = Parallel(n_jobs=-1)(delayed(process_file)(file_path) for file_path in tqdm(file_paths))

# Filtra i risultati None
results_list = [result for result in results_list if result is not None]

tca_count_df = pd.DataFrame(results_list)

  0%|          | 0/513 [00:00<?, ?it/s]

100%|██████████| 513/513 [11:12<00:00,  1.31s/it]


In [4]:
# set index to 'Cell_lines' and remove index name
tca_count_df = tca_count_df.set_index('Cell_lines')
tca_count_df.index.name = None

tca_count_df["TCA canonical"] = tca_count_df["TCA canonical"] / 10000
tca_count_df["TCA non canonical"] = tca_count_df["TCA non canonical"] / 10000
tca_count_df["Valid samples"] = tca_count_df["TCA canonical"] + tca_count_df["TCA non canonical"]
tca_count_df.head()

,TCA canonical,TCA non canonical,both active,biomass mean canonical,biomass mean non canonical,biomass mean both active,activity mean TCA canonical,activity mean TCA non canonical,Valid samples
22RV1_PROSTATE,0.1376,0.3547,84,0.047183,0.076068,0.043725,2.473520,14.794442,0.4923
2313287_STOMACH,0.1131,0.3046,41,0.058132,0.090999,0.094560,0.988751,13.072795,0.4177
5637_URINARY_TRACT,0.1265,0.5422,285,0.045263,0.054712,0.026731,5.801186,12.879744,0.6687
697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,0.1531,0.5623,188,0.046094,0.057049,0.033776,3.529251,10.430198,0.7154
769P_KIDNEY,0.0898,0.4553,218,0.036136,0.062512,0.018129,7.481572,17.514556,0.5451
